# Reaching multiple targets with a manipulator
The objective of this exercise is to reach multiple targets with a manipulator.

We provide a basic example for reaching one point, and you have to modify it for sequence of multiple targets. Below it is the basic example, there we'll guide you to the final result.


In [10]:
# %load arm_example.py
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data

robot = example_robot_data.load('talos_arm')
robot_model = robot.model

DT = 1e-3
T = 25
target = np.array([0.4, 0, 0.4])


# affichage du bras robotique
display = crocoddyl.GepettoDisplay(robot)

# affichage de la sphère rouge (marqueur de référence)
display.robot.viewer.gui.addSphere('world/point', .05, [1., 0., 0., 1.])  # radius = .1, RGBA=1001
display.robot.viewer.gui.applyConfiguration('world/point', target.tolist() + [0., 0., 0., 1.])  # xyz+quaternion

# actualisation : pour afficher les trucs juste au dessus
display.robot.viewer.gui.refresh()

# Create the cost functions
Mref = crocoddyl.FrameTranslation(robot_model.getFrameId("gripper_left_joint"), target) # Frame Translation entre main et arrivée (coordonnées de l'arrivée dans le repère de la main ?)
state = crocoddyl.StateMultibody(robot.model) # états du bras robotique
goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, Mref) # Cost entre l'état actuel du bras et la position désirée finale
xRegCost = crocoddyl.CostModelState(state) # Initialisation du modele de cout sur les états 0.5 x² par défaut
uRegCost = crocoddyl.CostModelControl(state) # Initialisation du modele de cout sur les commandes 0.5 u² par défaut

# Create cost model per each action model
runningCostModel = crocoddyl.CostModelSum(state) # Initialisation la somme de couts du model (en execution)
terminalCostModel = crocoddyl.CostModelSum(state) # Initialisation de la somme de couts du model (final)

# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, 1e2) # ajout de la fonction de cout goalTrackingCost (main du bras) au model "running" (exec) : poids 1e2
runningCostModel.addCost("stateReg", xRegCost, 1e-4) # ajout de la fonction de cout xRegCost (cout de l'état) au model running : poids 1e-4
runningCostModel.addCost("ctrlReg", uRegCost, 1e-7) # ajout de la fonction de cout uRegCost (cout de la commande) au model running : poids 1e-7
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e5) # ajout de la fonction de cout goalTrackingCost (main du bras) au model terminal : poids 1e5
terminalCostModel.addCost("stateReg", xRegCost, 1e-4) # ajout de la fonction de cout xRegCost (cout de l'état) au model terminal : poids de 1e-4
terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7) # ajout de la fonction de cout uRegCost (cout de la commande) au model terminal : poids de 1e-7

# Create the actuation model
actuationModel = crocoddyl.ActuationModelFull(state) # initialisation de l'actionnement du systeme

# Create the action model
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), DT) # creation du modele "runnning" en cours d'exec : état du systeme + model d'actuation + runningCostModel + DT=1e-3
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel)) # creation du modele "terminal" final
#runningModel.differential.armature = 0.2 * np.ones(state.nv)
#terminalModel.differential.armature = 0.2 * np.ones(state.nv)

# Create the problem
q0 = np.array([2., 1.5, -2., 0., 0., 0., 0.]) # joints initiaux TODO : à tester, modifier pour etre sur (parler que ça marche pas / comprends pas)
# q0 = np.array([0.2, 0.2, -0.2, 0., 0., 0., 0.]) # joints initiaux TODO : à tester, modifier pour etre sur

x0 = np.concatenate([q0, pinocchio.utils.zero(state.nv)]) # états initiaux : joints pos + joints vitesses
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel) # creation du probleme : etat initial, models * 25 (running model), model final (terminal model)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem) # creation du DDP solver pour résoudre le probleme
# ddp.setCallbacks([crocoddyl.CallbackVerbose()]) # set callbacks pour des logs
ddp.setCallbacks([crocoddyl.CallbackDisplay(display)]) # voir la résolution du probleme sur gepetto-gui

# Solving it with the DDP algorithm
ddp.solve() # le DDP résout le probleme

# Visualizing the solution in gepetto-viewer
display.displayFromSolver(ddp) # visualisation de la solution finale dans la simu

robot_data = robot_model.createData() # creation des données à partir du model contient l'état actuel du bras pour afficher les coordonnées dans un print()
xT = ddp.xs[-1]
print(state.nq)
pinocchio.forwardKinematics(robot_model, robot_data, xT[:state.nq])
pinocchio.updateFramePlacements(robot_model, robot_data)
print('Finally reached = ', robot_data.oMf[robot_model.getFrameId("gripper_left_joint")].translation.T)

7
Finally reached =  [0.38255359 0.01645868 0.3964032 ]


### II.b State cost
In this part of the tutorial you must define a State model. It defines 
 - the dimension of the state and its tangent, and
 - the exponential/integrate and difference/log operators.
The operators can described using Pinocchio functions. And the exercite consists on adding them into your State class. Please note crocoddyl has abstract functions for this.

The state cost uses a reference in state space (State.zero() by default). The cost is the distance, computed with state.difference between the current state and the reference. Hence, with this cost, we regularize both position and velocity.

### II.c Control cost

The control cost uses a control reference as in the state cost. The cost is the distance the current control and the reference. Hence the cost regularizes torque commands.

## I. DifferentialActionModel for Pinocchio ABA
This scenario uses an action model that computes 2nd order differential dynamics with Pinocchio. Note that it can accept several cost models. This action model is tailored for robot applications, and at the same time, it's modular since:
 - you can modify the robot dynamics by changing Pinocchio model, and
 - you can formulate any cost function by simply adding running and terminal costs.

## II. Cost models

A cost model computes a scalar cost value and its gradient and Hessian. All the models implemented are computing a cost residual and are computing the Hessian with the Gauss approximation.

We implemented reusable cost models for controlling 
 - a frame placement (translation or velocity),
 - the center of mass position, and 
 - state  and control spaces.

In the example above, we used the CostModelFrameTranslation which defines a 3d position task, and the state and control regularizers.

As for any cost model in crocoddyl, if you write your own cost model you need to create a data class for your cost function. The cost data must be created from a pinocchio data (the rational is that the pinocchio data used to compute the dynamics should be re-used to compute the cost).


In [11]:
dataCollector = crocoddyl.DataCollectorMultibody(robot.data)
trackData = goalTrackingCost.createData(dataCollector)

### II.a Frame position cost

You define a frame ID and the reference position as a 3D array. The cost is the distance between the frame and the target. This cost depends on $\mathbf{x}$ (specifically the configuration $\mathbf{q}$). You can double check the 0s in its gradient.

In [12]:
pinocchio.updateFramePlacements(robot.model, robot.data)
pinocchio.computeJointJacobians(robot.model, robot.data, xT[:state.nq])
goalTrackingCost.calc(trackData, x0)
goalTrackingCost.calcDiff(trackData, x0)
print(trackData.Lx, trackData.Lu) # print Jacobian of cost functions (x et u) : matrice de derivée 1er ordre. [valeurs pour les angles moteur contribuant à configuration q, 6 premiers, pas le dernier(moteur doigts)]
# les autres c'est les vitesses des moteurs et l'autre tableau c'est la commande (torque) donc y'a des zéros

[-0.0053234  -0.00557997 -0.00192322  0.00066331 -0.00079466 -0.0006631
  0.          0.          0.          0.          0.          0.
  0.          0.        ] [0. 0. 0. 0. 0. 0. 0.]


### II.d Add cost models to the differential action model
Each time we want to include a new cost function, we use addCost function inside our DAM. In this function you're also able its weight. (DAM = Differential Action Model ?)

## III. Create the problem with integrated action model
Differential action models describe cost and dynamics in continuous-time, however our optimal control solvers work in discrete-time. We have created the integrated action model in order to deal with this.

In the previous code, we have used an abstract class that uses simpletic Euler rules. In the cartpole exercise you have learnt how to use integrated action models for your problem.

## IV. Callbacks

Callback functions are needed for analysing and debugging the performance of the solver for your specific problem.
For problems defined with Pinocchio, you can display the robot trajectory per each iterate by including CallbackDisplay. With this callback, you can display robot motions with different rates. Additionally, CallbackVerbose prints a message that allows us to understand the behaviour of the solver.

Generally speaking, an user is able to describe any callback function. This function will be run once per iterate and it has access to all data.

## VI. Modifying the example

Start by defining several targets (let's say 4 targets, all at x=0.4, and at y and z being either 0 or 0.4), and display then in the viewer.


In [13]:
targets = np.array([[0.4, 0., 0.4],
                    [0.4, 0.4, 0.4],
                    [0.4, 0.4, 0.],
                    [0.4, 0., 0.]])
# target1 = np.array([0.4, 0., 0.4])
# target2 = np.array([0.4, 0.4, 0.])
# target3 = np.array([0.4, 0.4, 0.4])
# target4 = np.array([0.4, 0., 0.])

display.robot.viewer.gui.addSphere('world/point1', .05, [1., 0., 0., 1.])  # radius = .1, RGBA=1001
display.robot.viewer.gui.applyConfiguration('world/point1', targets[0,:].tolist() + [0., 0., 0., 1.])  # xyz+quaternion

display.robot.viewer.gui.addSphere('world/point2', .05, [1., 0., 0., 1.])  # radius = .1, RGBA=1001
display.robot.viewer.gui.applyConfiguration('world/point2', targets[1,:].tolist() + [0., 0., 0., 1.])  # xyz+quaternion

display.robot.viewer.gui.addSphere('world/point3', .05, [1., 0., 0., 1.])  # radius = .1, RGBA=1001
display.robot.viewer.gui.applyConfiguration('world/point3', targets[2,:].tolist() + [0., 0., 0., 1.])  # xyz+quaternion

display.robot.viewer.gui.addSphere('world/point4', .05, [1., 0., 0., 1.])  # radius = .1, RGBA=1001
display.robot.viewer.gui.applyConfiguration('world/point4', targets[3,:].tolist() + [0., 0., 0., 1.])  # xyz+quaternion

display.robot.viewer.gui.refresh()

The shooting problem will be composed of 4 sequences of action models. Each sequence consists on T shooting "running" nodes and 1 terminal node. The running nodes mostly have regularization terms, while the terminal nodes have a strong cost toward the respective target.

[ R1,R1,R1 ... R1,T1, R2,R2 .... R2, T2, R3 ... R3, T3, R4 ... R4 ] , T4

First create 4 running models and 4 terminal models.

In [14]:
state = crocoddyl.StateMultibody(robot.model)

runningCostModels = [crocoddyl.CostModelSum(state)] * 4
terminalCostModels = [crocoddyl.CostModelSum(state)] * 4


Then you need to add a position cost, and state and control regularization to each running action model. Please  note that for terminal action model is only needed the position cost. Additionally, in the running models, the position cost should be low, and it should be high in the terminal models.

In [15]:
Mref0 = crocoddyl.FrameTranslation(robot_model.getFrameId("gripper_left_joint"), targets[0,:])
goalTrackingCost0 = crocoddyl.CostModelFrameTranslation(state, Mref0)
xRegCost0 = crocoddyl.CostModelState(state)
uRegCost0 = crocoddyl.CostModelControl(state)

Mref1 = crocoddyl.FrameTranslation(robot_model.getFrameId("gripper_left_joint"), targets[1,:])
goalTrackingCost1 = crocoddyl.CostModelFrameTranslation(state, Mref1)
xRegCost1 = crocoddyl.CostModelState(state)
uRegCost1 = crocoddyl.CostModelControl(state)

Mref2 = crocoddyl.FrameTranslation(robot_model.getFrameId("gripper_left_joint"), targets[2,:])
goalTrackingCost2 = crocoddyl.CostModelFrameTranslation(state, Mref2)
xRegCost2 = crocoddyl.CostModelState(state)
uRegCost2 = crocoddyl.CostModelControl(state)

Mref3 = crocoddyl.FrameTranslation(robot_model.getFrameId("gripper_left_joint"), targets[3,:])
goalTrackingCost3 = crocoddyl.CostModelFrameTranslation(state, Mref3)
xRegCost3 = crocoddyl.CostModelState(state)
uRegCost3 = crocoddyl.CostModelControl(state)

runningCostModels[0].addCost("gripperPose0", goalTrackingCost0, 1e5) # 1e2
runningCostModels[1].addCost("gripperPose1", goalTrackingCost1, 1e5)
runningCostModels[2].addCost("gripperPose2", goalTrackingCost2, 1e5)
runningCostModels[3].addCost("gripperPose3", goalTrackingCost3, 1e5)

runningCostModels[0].addCost("stateReg0", xRegCost0, 1e-4)
runningCostModels[1].addCost("stateReg1", xRegCost1, 1e-4)
runningCostModels[2].addCost("stateReg2", xRegCost2, 1e-4)
runningCostModels[3].addCost("stateReg3", xRegCost3, 1e-4)

runningCostModels[0].addCost("ctrlReg0", uRegCost0, 1e-7)
runningCostModels[1].addCost("ctrlReg1", uRegCost1, 1e-7)
runningCostModels[2].addCost("ctrlReg2", uRegCost2, 1e-7)
runningCostModels[3].addCost("ctrlReg3", uRegCost3, 1e-7)

terminalCostModels[0].addCost("gripperPose0", goalTrackingCost0, 1e2) # 1e5
terminalCostModels[1].addCost("gripperPose1", goalTrackingCost1, 1e2)
terminalCostModels[2].addCost("gripperPose2", goalTrackingCost2, 1e2)
terminalCostModels[3].addCost("gripperPose3", goalTrackingCost3, 1e6)
# terminalCostModels[1].addCost("gripperPose1", goalTrackingCost1, 1e9)
# terminalCostModels[2].addCost("gripperPose2", goalTrackingCost2, 1e12)
# terminalCostModels[3].addCost("gripperPose3", goalTrackingCost3, 1e15)

terminalCostModels[0].addCost("stateReg0", xRegCost0, 1e-4)
terminalCostModels[1].addCost("stateReg1", xRegCost1, 1e-4)
terminalCostModels[2].addCost("stateReg2", xRegCost2, 1e-4)
terminalCostModels[3].addCost("stateReg3", xRegCost3, 1e-4)

terminalCostModels[0].addCost("ctrlReg0", uRegCost0, 1e-7)
terminalCostModels[1].addCost("ctrlReg1", uRegCost1, 1e-7)
terminalCostModels[2].addCost("ctrlReg2", uRegCost2, 1e-7)
terminalCostModels[3].addCost("ctrlReg3", uRegCost3, 1e-7)

actuationModel = crocoddyl.ActuationModelFull(state)

runningModels = [None] * 4
runningModels[0] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModels[0]), DT)
runningModels[1] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModels[1]), DT)
runningModels[2] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModels[2]), DT)
runningModels[3] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModels[3]), DT)

terminalModels = [None] * 4
terminalModels[0] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModels[0]))
terminalModels[1] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModels[1]))
terminalModels[2] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModels[2]))
terminalModels[3] = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModels[3]))

Now create a shooting problem.

In [16]:
seq0 = [runningModels[0]]*T + [terminalModels[0]]
seq1 = [runningModels[1]]*T + [terminalModels[1]]
seq2 = [runningModels[2]]*T + [terminalModels[2]]
seq3 = [runningModels[3]]*T 
# problem = crocoddyl.ShootingProblem(x0,[runningModels[0]]*T ,terminalModels[0])
# problem = crocoddyl.ShootingProblem(x0,seq0+seq1,terminalModels[1])
# problem = crocoddyl.ShootingProblem(x0,seq0+seq1+seq2,terminalModels[2])
problem = crocoddyl.ShootingProblem(x0,seq0+seq1+seq2+seq3,terminalModels[3])

Create a DDP solver for this problem and run it. 

In [17]:
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackDisplay(display)]) # set callbacks pour des logs
ddp.solve()
display.displayFromSolver(ddp) # visualisation de la solution finale dans la simu

Well, it should not work, at least no on the first shot. The DDP solver is likely not strong enough to accept the random weights that you have selected. 

If it is working nicely from the first shot, display it in the viewer and go take a coffee. But you will likely have to tweak the gains to make it work.

**It is suggested to first optimize only sequence 1. When you are happy with it, add sequence 2 and optimize again, etc.**


## V. Penalty
The solver works with double precisions, so it is quite robust to high weight. 10000 is likely to be accepted for example. But if you make the problem too difficult, the solver will break. 

In that case, you can implement a simple penalty solver by setting the weight to be 10**i, and creating a for loop to explore i from 0 to 5. At each iteration of the loop, run the solver from the previous solution and for few iterations only.

In [18]:
for i in range(1,6):
    for m in terminalModels:
        m.costs.costs['gripperPose'].weight = 10**i
    ddp.solve(ddp.xs, ddp.us, 10)

AttributeError: 'IntegratedActionModelEuler' object has no attribute 'costs'